# Avaliação I - Ciência de Dados

Notebook gerado automaticamente com a análise dos dados enviados.
### Nome: Luiz Fernando Mendes Alberton
**Passos incluídos**: carregamento, limpeza, análise descritiva, visualizações e salvamento de dados limpos.


In [4]:
# Carregamento dos dados
import pandas as pd
import os

# Usar o arquivo disponível no diretório atual
csv_path = 'funcionarios.csv'  # Arquivo encontrado no diretório atual

if os.path.exists(csv_path):
    print(f"Carregando arquivo de: {csv_path}")
    df = pd.read_csv(csv_path, low_memory=False)
    print(f"Dados carregados com sucesso! Shape: {df.shape}")
else:
    print(f"Arquivo não encontrado: {csv_path}")
    print("Diretório atual:", os.getcwd())
    print("Arquivos disponíveis:")
    print([f for f in os.listdir('.') if f.endswith('.csv')])

Carregando arquivo de: funcionarios.csv
Dados carregados com sucesso! Shape: (5, 4)


In [5]:
df.head(10)

,Nome,Idade,Salário,Cidade
0,Ana,25,5000,São Paulo
1,Bruno,30,7500,Rio de Janeiro
2,Carlos,35,7000,Belo Horizonte
3,Diana,28,5800,São Paulo
4,Eduardo,32,6200,Rio de Janeiro


In [8]:
# Informações gerais
print('Colunas:', df.columns.tolist())
print('\nNulos por coluna (percentual):')
print((df.isna().mean()*100).round(2).sort_values(ascending=False).head(20))


Colunas: ['Nome', 'Idade', 'Salário', 'Cidade']

Nulos por coluna (percentual):
Nome       0.0
Idade      0.0
Salário    0.0
Cidade     0.0
dtype: float64


In [ ]:
# Limpeza básica (adaptada para dados de funcionários)
import re

def parse_valor(x):
    if pd.isna(x):
        return float('nan')
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip().replace('R$','').replace('r$','').replace('.','').replace(',','.')
    s = re.sub(r'[^0-9\.-]', '', s)
    try:
        return float(s)
    except:
        return float('nan')

def parse_parcelas(x):
    if pd.isna(x):
        return float('nan')
    if isinstance(x, (int, float)):
        return int(x)
    s = str(x).strip()
    import re
    m = re.search(r"(\d+)", s)
    if m:
        return int(m.group(1))
    return float('nan')

# Aplicar limpeza para as colunas disponíveis
if 'valor' in df.columns:
    df['valor_clean'] = df['valor'].apply(parse_valor)
if 'valor_liquido' in df.columns:
    df['valor_liquido_clean'] = df['valor_liquido'].apply(parse_valor)
if 'parcelas' in df.columns:
    df['parcelas_clean'] = df['parcelas'].apply(parse_parcelas)

# Para dados de funcionários, vamos limpar o salário se necessário
if 'Salário' in df.columns:
    df['salario_clean'] = df['Salário'].apply(parse_valor)

# converter data
if 'data' in df.columns:
    df['data_dt'] = pd.to_datetime(df['data'], dayfirst=False, errors='coerce')

print('Limpeza aplicada. Colunas disponíveis:')
print('Colunas originais:', df.columns.tolist())
print('\nPrimeiras linhas:')
print(df.head())

Limpeza aplicada. Exemplo:


KeyError: "None of [Index(['valor', 'valor_clean', 'parcelas', 'parcelas_clean', 'data',\n       'data_dt'],\n      dtype='object')] are in the [columns]"

In [9]:
# Estatísticas descritivas
print('shape:', df.shape)
print('\nResumo valor_clean:')
print(df['valor_clean'].describe())

# Histograma (gerar figura)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,5)
plt.hist(df['valor_clean'].dropna(), bins=40)
plt.title('Histograma de valor_clean')
plt.xlabel('Valor')
plt.ylabel('Frequência')
plt.show()

# Faturamento por mês
if 'data_dt' in df.columns:
    monthly = df.dropna(subset=['data_dt','valor_clean']).set_index('data_dt').resample('M')['valor_clean'].sum()
    display(monthly)
    plt.figure()
    monthly.plot(marker='o')
    plt.title('Faturamento por mês')
    plt.ylabel('Soma valor_clean')
    plt.xlabel('Mês')
    plt.xticks(rotation=45)
    plt.show()


shape: (5, 4)

Resumo valor_clean:


KeyError: 'valor_clean'

In [10]:
# Segmentações de negócio (exemplos)
if 'categoria_lojista' in df.columns:
    print('\nTop categorias por faturamento:')
    display(df.groupby('categoria_lojista')['valor_clean'].sum().sort_values(ascending=False).head(10))

if 'metodo_pagamento' in df.columns and 'status' in df.columns:
    s = df.copy()
    s['status_norm'] = s['status'].str.lower().str.strip()
    grouped = s.groupby('metodo_pagamento').agg(
        total = ('status_norm','count'),
        approved = ('status_norm', lambda x: (x=='approved').sum()),
        denied = ('status_norm', lambda x: (x=='denied').sum())
    )
    grouped['tx_aprov'] = (grouped['approved']/grouped['total']).round(3)
    display(grouped.sort_values('total', ascending=False))


In [11]:
# Salvar versão limpa
out_path = 'aed_fintech_clean.csv'
cols_to_keep = [c for c in df.columns if c.endswith('_clean') or c in ['id_transacao','id_cliente','data_dt','cidade','estado','categoria_lojista','metodo_pagamento','status']]
if cols_to_keep:
    df[cols_to_keep].to_csv(out_path, index=False)
    print('Arquivo limpo salvo em', out_path)
else:
    print('Nenhuma coluna limpa identificada para salvar.')


Nenhuma coluna limpa identificada para salvar.


In [12]:
# Visualizações adicionais
import matplotlib.pyplot as plt
import os

# Gerar visualizações diretamente no notebook
plt.rcParams['figure.figsize'] = (12,8)

# 1. Histograma de valores (se ainda não foi mostrado)
if 'valor_clean' in df.columns:
    plt.figure(figsize=(10,6))
    plt.hist(df['valor_clean'].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Distribuição de Valores das Transações')
    plt.xlabel('Valor (R$)')
    plt.ylabel('Frequência')
    plt.grid(True, alpha=0.3)
    plt.show()

# 2. Top categorias por faturamento (gráfico de barras)
if 'categoria_lojista' in df.columns and 'valor_clean' in df.columns:
    top_cat = df.groupby('categoria_lojista')['valor_clean'].sum().sort_values(ascending=False).head(8)
    plt.figure(figsize=(12,6))
    top_cat.plot(kind='barh', color='lightcoral')
    plt.title('Top Categorias por Faturamento Total')
    plt.xlabel('Faturamento Total (R$)')
    plt.ylabel('Categoria')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# 3. Distribuição por método de pagamento
if 'metodo_pagamento' in df.columns:
    payment_counts = df['metodo_pagamento'].value_counts()
    plt.figure(figsize=(10,6))
    payment_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title('Distribuição por Método de Pagamento')
    plt.ylabel('')
    plt.show()

print("Visualizações geradas com sucesso!")

Visualizações geradas com sucesso!
